In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

np.random.seed(42)
n_samples = 1000

# Parameters we want our NLL to eventually "discover"
true_pi = 0.3   # 30% chance of a structural zero (e.g., closed location)
true_mu = 5.0   # Average traffic when active
true_n = 2.0    # 'n' parameter for NegBinom (related to dispersion)
true_p = true_n / (true_n + true_mu) # probability of success for scipy

# Generate the data
# 1. Decide if it's a structural zero
is_structural_zero = np.random.rand(n_samples) < true_pi

# 2. Generate counts from Negative Binomial
counts = stats.nbinom.rvs(n=true_n, p=true_p, size=n_samples)

# 3. Combine them: if structural zero, count is 0; else keep count
mobility_counts = np.where(is_structural_zero, 0, counts)

print(f"First 10 counts: {mobility_counts[:10]}")

In [ ]:
def zinb_nll(params, y):
    # Unpack parameters (we'll optimize these)
    mu, n, pi = params

    # Constraint checks (optional but good practice):
    # mu > 0, n > 0, 0 <= pi <= 1
    if mu <= 0 or n <= 0 or pi < 0 or pi > 1:
        return np.inf

    # Convert mu/n to p for scipy
    p = n / (n + mu)

    # 1. Create the mask
    is_zero = (y == 0)

    # 2. Calculate Probability for Zeros
    prob_zeros = pi + (1 - pi) * stats.nbinom.pmf(y[is_zero], n, p)

    # 3. Calculate Probability for Non-Zeros
    prob_non_zeros = (1 - pi) * stats.nbinom.pmf(y[~is_zero], n, p)

    # 4. Compute Negative Log-Likelihood
    nll = -np.sum(np.log(prob_zeros + 1e-10)) - np.sum(np.log(prob_non_zeros + 1e-10))

    return nll

In [ ]:
from scipy.optimize import minimize

result = minimize(zinb_nll, x0=[3, 3, 0.5], args=(mobility_counts,))

print(f"Recovered Parameters: mu={result.x[0]:.2f}, n={result.x[1]:.2f}, pi={result.x[2]:.2f}")